In [ ]:
from newspaper import Article #newspaper has some nlp stuff, don't know if it is good enough

import spacy
from spacy import displacy
from collections import Counter
import en_core_web_sm
nlp = en_core_web_sm.load()

In [37]:
#https://towardsdatascience.com/scrape-and-summarize-news-articles-in-5-lines-of-python-code-175f0e5c7dfc
url = "https://thenewstack.io/dont-mess-with-the-master-working-with-branches-in-git-and-github/"
article = Article(url, language='en', fetch_images=False)

#download and parse the article
article.download()
article.parse()
#article.nlp() #Don't know if we need to parse

text = article.text

In [39]:
doc = nlp(text)
#displacy.render(nlp(str(sentences[20])), jupyter=True, style='ent')

labels = [x.label_ for x in doc.ents]
Counter(labels)

Counter({'ORG': 14,
         'GPE': 2,
         'CARDINAL': 11,
         'NORP': 4,
         'ORDINAL': 2,
         'WORK_OF_ART': 3,
         'PERSON': 1})

In [41]:
sentences = [x for x in doc.sents]
print(sentences[21])

Instead, everyone uses branches created from master to experiment, make edits and additions and changes, before eventually rolling that branch back into the master once they have been approved and are known to work.


In [43]:
items = [x.text for x in doc.ents]
Counter(items).most_common(3)

[('hello_octo', 10), ('GitHub', 9), ('one', 3)]